In [ ]:
# Generate model output for validation
X_val = torch.from_numpy(X_train[val_index]).permute(0, 3, 1, 2).float()

# Load fold's model
model = CoolSystem(hparams)
model.load_state_dict(
    torch.load(
        os.path.join(
            hparams.log_dir, "2022_09_13_09:40:01", "fold=0-epoch=10-val_loss=2.0033-val_roc_auc=0.6759.ckpt"
        )
    )["state_dict"]
)

preds = model(X_val)

# Create activation output
log_softmax = torch.nn.LogSoftmax(dim=-1)

# Convert raw output to probabilities
preds = np.exp(log_softmax(preds).detach().numpy())

# Create df with img paths and predicted label probs
scores_df = pd.DataFrame(preds, columns=val_data.columns[1:])
scores_df = pd.merge(
    metadata.iloc[val_index, 1:3].reset_index(drop=True),
    scores_df, 
    left_index=True,
    right_index=True
)
# Write predictions to log
scores_df.to_csv(
    os.path.join(hparams.log_dir, hparams.log_name, f"{hparams.log_name}_preds_fold_{fold_i}.csv"),
    index=False
)

# Cleanup
del model
gc.collect()
torch.cuda.empty_cache()
